## Model Architecture

_All convolution layers have `3 * 3` kernel,  `padding=1`, batch normalization and relu activation_

__ResidualBlock:__ _in_channel, out_channel, stride_

- Conv(in_channel, out_channel, stride)
- Conv(out_channel, out_channel, stride=1)

`*` This block has a residual connection. To match dimmensions of output and residual use `1 * 1` convolution and stride. 

__ResidualLayer:__ _in_channel, out_channel, stride_

- ResidualBlock(in_channel, out_channel, stride)
- ResidualBlock(out_channel, out_channel, stride=1)

__ResidualNetwork__:
- Conv(3, 64, stride=1)
- ResidualLayer(64, 64, stride=1)
- ResidualLayer(64, 128, stride=2)
- ResidualLayer(128, 256, stride=2)
- ResidualLayer(256, 512, stride=2)
- AveragePool(4, 4)
- Linear(512, 10)

# importing

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np
import datetime





# Hyper Parameter

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
batch_size_train = 128
batch_size_test =  128
learning_rate = 0.001
n_eopchs = 10


# Loading data

In [ ]:
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR10(root='./data',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data',
                                            train=False, 
                                            transform=transforms.ToTensor())

trainloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=128, 
                                           shuffle=True)

testloader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=128, 
                                          shuffle=False)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

for images, labels in trainloader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break


Files already downloaded and verified
Image batch dimensions: torch.Size([128, 3, 32, 32])
Image label dimensions: torch.Size([128])


#  Network Design

In [ ]:
class block(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.model = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU(inplace = True),
                            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                            nn.BatchNorm2d(out_channels)
                              )
        self.residual = None
        if stride > 1:
            self.residual = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                            nn.BatchNorm2d(out_channels)
                              )
        self.relu = nn.ReLU(inplace = True)

    def forward(self, x):
        residual = x
        out = self.model(x)
        if self.residual is not None:
            residual = self.residual(x)

        out += residual
        out = self.relu(out)
        return out

################################################################################
class ResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=True)

        self.layer1 = self.make_layer(in_channel=64, out_channel=64, stride=1)
        self.layer2 = self.make_layer(in_channel=64, out_channel=128, stride=2)
        self.layer3 = self.make_layer(in_channel=128, out_channel=256, stride=2)
        self.layer4 = self.make_layer(in_channel=256, out_channel=512, stride=2)

        self.avgPool = nn.AvgPool2d(kernel_size=4)
        self.fc = nn.Linear(512, 10)



    def make_layer(self, in_channel, out_channel, stride):
        model = nn.Sequential(
            block(in_channel, out_channel, stride),
            block(out_channel, out_channel, stride=1)
        )
        return model

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgPool(x)

        x = x.reshape(x.shape[0], 512)
        x = self.fc(x)

        return x

# Optimization Algorithm:

In [ ]:
model = ResNet().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training

In [ ]:
#######################################################
# TODO: Feed the inputs data to the MLP network and   #
# optimize Cross-Entropy loss by using target labels. #
# Then update weights and biases.                     #
#######################################################

losses = []
acc  = [[] for x in range(10)]
acc[0].append(10)

num_epochs=5
num_batchs = len(trainloader)
device = 'cpu'
freq = 100
for epoch in range(num_epochs):
    print('[Epoch]', epoch)
    total_train=0
    correct_train=0
    running_loss = 0.0
    for batch, data in enumerate(trainloader, 0):

        inputs, labels = data

        # zero the parameter gradients:
        optimizer.zero_grad()

        # forward pass:
        outputs = model.forward(inputs)

        # backward pass:
        loss = criterion(outputs, labels)
        loss.backward()

        # optimization:
        optimizer.step()

        if batch % freq == 0:
            print('     Loss: {:.3f} '.format(loss.item()))
    ##############################
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    loss = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            outputs = model(images.to(device))
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels.to(device)).squeeze()
            loss += criterion(outputs, labels)
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    loss /= len(testloader)
    losses.append(loss)
    print('Loss in Epoch:',loss)
    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))
        acc[i].append(100 * class_correct[i] / class_total[i])


[Epoch] 0
     Loss: 1.391 
     Loss: 1.408 
     Loss: 0.992 
     Loss: 1.127 
Loss in Epoch: tensor(1.0642)
Accuracy of plane : 68 %
Accuracy of   car : 89 %
Accuracy of  bird : 45 %
Accuracy of   cat : 64 %
Accuracy of  deer : 40 %
Accuracy of   dog : 24 %
Accuracy of  frog : 77 %
Accuracy of horse : 56 %
Accuracy of  ship : 90 %
Accuracy of truck : 74 %
[Epoch] 1
     Loss: 0.828 
     Loss: 1.042 
     Loss: 0.992 
     Loss: 0.766 
Loss in Epoch: tensor(0.8385)
Accuracy of plane : 86 %
Accuracy of   car : 85 %
Accuracy of  bird : 57 %
Accuracy of   cat : 38 %
Accuracy of  deer : 44 %
Accuracy of   dog : 66 %
Accuracy of  frog : 69 %
Accuracy of horse : 72 %
Accuracy of  ship : 87 %
Accuracy of truck : 84 %
[Epoch] 2
     Loss: 0.760 
     Loss: 0.729 
     Loss: 0.829 
     Loss: 0.688 
Loss in Epoch: tensor(0.6717)
Accuracy of plane : 89 %
Accuracy of   car : 96 %
Accuracy of  bird : 75 %
Accuracy of   cat : 52 %
Accuracy of  deer : 66 %
Accuracy of   dog : 66 %
Accuracy of  f

In [ ]:
freq = 250
def train(model, epoch):
    model = model.train()
    print('[Epoch]', epoch)
    for batch, (inputs, labels) in enumerate(trainloader):  
        inputs = (inputs / 255).float().to(device)
        labels = labels.to(device)
        #training
        optimizer.zero_grad()
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        #end of traingning
        if batch % freq == 0:
            print('     Loss: {:.3f} '.format(loss.item()))


def test(model):
    model = model.eval()
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for (images, labels) in testloader:
            inputs = inputs.to(device)/255
            labels = labels.to(device)
            outputs = model(images.to(device))
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels.to(device)).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    print('[Epoch]',epoch + 1)
    for i in range(10):
        print('        Accuracy of %5s : %2d %%' % (classes[i], 100 * class_correct[i] / class_total[i]))



for epoch in range(1, n_eopchs):
    train(model, epoch)
    test(model)

[Epoch] 1
     Loss: 2.411 


KeyboardInterrupt: ignored

 in epoch  1
[Epoch] 1
        Accuracy of plane : 79 %
        Accuracy of   car : 62 %
        Accuracy of  bird : 39 %
        Accuracy of   cat : 37 %
        Accuracy of  deer : 36 %
        Accuracy of   dog : 46 %
        Accuracy of  frog : 69 %
        Accuracy of horse : 78 %
        Accuracy of  ship : 70 %
        Accuracy of truck : 82 %
 in epoch  2
[Epoch] 2
        Accuracy of plane : 81 %
        Accuracy of   car : 81 %
        Accuracy of  bird : 60 %
        Accuracy of   cat : 46 %
        Accuracy of  deer : 61 %
        Accuracy of   dog : 46 %
        Accuracy of  frog : 71 %
        Accuracy of horse : 75 %
        Accuracy of  ship : 66 %
        Accuracy of truck : 93 %
 in epoch  3


KeyboardInterrupt: ignored